In [1]:
%run imports
from src.functions import *
from src.classes import *
from src.plots import *
#195633320791780608, 195633325090480896


In [2]:
g166 = G166()

# Access and print the name and SkyCoord
print(g166.name)
print(g166.radius)
print(g166.SkyCoord)


G166
60 arcmin
<SkyCoord (ICRS): (ra, dec) in deg
    (81.625, 42.93333333)>


In [3]:
Star(195633320791780608).info
Star(195633325090480896).info

RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc"
float64,float64,float64,float64,float64,int32,str12,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str16,str21,float64,float32,float32,float32,SkyCoord
81.62564389768,42.93974983019,0.0447,0.0294,0.0004,--,,195633325090480896,3138.43115000,0.2922,0.0544,1.408,-2.592,0.064,0.046,1.046,6047.5,4.4779,16.263456,1.037928,16.644405,15.606478,--,--,2643.20166000,3779.71533000,5882.52580,3.53,3004,17.82,4535,16.2,,,0.002831,0.00702,0.005416,0.01244,"81.62564389768,42.93974983019,3138.43115"


In [3]:
coord = SkyCoord.from_name("SNR G166.0+04.3")
radius = 1*u.deg
# Gets all conical stars in the 1 deg region around G166 center:
# stars_in_region = get_stars_in_region(coord, radius)
stars_in_region = Table.read('data/stars_near_g166.ecsv', format='ascii.ecsv')
# stars_in_region
dist_mask_u = stars_in_region['rgeo']<5*u.kpc
dist_mask_l = stars_in_region['rgeo']>2.5*u.kpc
ruwe_mask = stars_in_region['RUWE']<1.4
plxq_mask = stars_in_region['Plx']/stars_in_region['e_Plx'] > 5
sir = stars_in_region[dist_mask_l & dist_mask_u & ruwe_mask & plxq_mask]
sir

RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc"
float64,float64,float64,float64,float64,int32,str11,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str1,str1,float64,float32,float32,float32,SkyCoord
81.81732252743,42.29640171072,0.0209,0.0133,0.652460,25508,2917-1532-1,195365594010029440,3889.14697000,0.2263,0.0240,-0.068,-0.170,0.028,0.019,1.183,--,--,9.356865,0.614367,9.571415,8.957047,--,--,3581.26050000,4326.85107000,3405656.34023,1399,2.027e+06,2206,2.072e+06,1834,--,--,0.002791,0.00303,0.0039,0.00693,"81.81732252743,42.29640171072,3889.14697"
81.07020714699,42.30666259738,0.0183,0.0114,0.747915,--,2917-102-1,195052027036610048,3563.87891000,0.2405,0.0219,0.160,-0.218,0.025,0.020,1.000,--,--,9.598747,2.372551,10.878381,8.505830,-9.57,0.15,3299.47510000,3859.00171000,2725509.90051,981,6.082e+05,643.1,3.139e+06,1333,--,--,0.002783,0.003017,0.003807,0.006824,"81.07020714699,42.30666259738,3563.87891"
80.28360077127,43.07576252577,0.0158,0.0105,0.991247,--,2917-2117-1,207333812437636608,3556.88159000,0.2495,0.0192,0.476,-0.550,0.023,0.016,1.126,8396.9,2.1345,9.989893,0.940436,10.362233,9.421797,2.47,0.31,3336.46069000,3824.27417000,1901032.94385,731,9.784e+05,481.5,1.35e+06,309.4,--,--,0.002787,0.002841,0.003788,0.006628,"80.28360077127,43.07576252577,3556.88159"
81.65634289596,43.01971193381,0.0158,0.0105,0.089374,--,2917-195-1,195645832035222400,3955.39429000,0.2262,0.0176,0.138,0.311,0.024,0.017,0.977,18496.6,3.5290,10.021754,0.412804,10.157121,9.744317,--,--,3735.79517000,4203.52783000,1846057.12511,552.7,1.182e+06,484.6,1.003e+06,250.5,--,--,0.002774,0.002825,0.003789,0.006615,"81.65634289596,43.01971193381,3955.39429"
82.35367329679,43.11650908429,0.0149,0.0114,0.563308,--,--,195474273863568000,4166.56787000,0.2086,0.0188,0.288,-0.305,0.021,0.015,0.931,7299.5,1.9519,10.153550,0.803403,10.460058,9.656655,-5.57,0.27,3871.85132000,4515.83301000,1635033.04191,571,8.941e+05,316.4,1.088e+06,258.7,--,--,0.002781,0.002816,0.003788,0.006605,"82.35367329679,43.11650908429,4166.56787"
81.00295672003,42.41899671892,0.0178,0.0112,0.688246,--,2917-1525-1,195241619775197824,2617.66113000,0.3361,0.0213,0.857,-2.544,0.028,0.020,1.045,--,--,10.179756,2.526001,11.567115,9.041114,4.48,0.14,2481.29810000,2749.68115000,1596042.10640,540,3.225e+05,436.6,1.917e+06,874.7,--,--,0.00278,0.003154,0.003812,0.006965,"81.00295672003,42.41899671892,2617.66113"
80.86337826456,42.39159306647,0.0187,0.0115,0.779197,--,2917-1551-1,195243097243949184,3730.47852000,0.2229,0.0206,0.123,-0.207,0.026,0.019,1.055,--,--,10.262775,2.427823,11.580130,9.152307,-9.62,0.21,3451.86206000,4004.74097000,1478552.08385,431.1,3.187e+05,407.9,1.731e+06,875,--,--,0.002773,0.003117,0.003819,0.006936,"80.86337826456,42.39159306647,3730.47852"
80.65165660438,42.40117278340,0.0152,0.0101,0.891857,--,2917-2411-1,195232003349318144,2570.28589000,0.3565,0.0163,-0.239,-1.693,0.020,0.017,1.027,--,--,10.334803,1.374884,10.950708,9.575825,-34.79,0.36,2452.75879000,2702.69653000,1383648.69411,213.7,5.69e+05,220.4,1.172e+06,256,--,--,0.00276,0.002822,0.003787,0.006608,"80.65165660438,42.4011727834,2570.28589"


# Optimise matching

- trace all stars back for 20 kyr
- create one config file for each star. 4749 config files needed. each config file performs the trace for one star with all possible matches.
- need to create these traceback config files.

In [11]:
star_lines = fs4giesler(sir)
star_lines #this is the input.

TypeInput,RA,DE,Plx,e_Plx,RV,e_RV,RVdist,pmRA,e_pmRA,pmDE,e_pmDE,Source
,,,mas,mas,,,,mas / yr,mas / yr,mas / yr,mas / yr,
int64,str12,str12,float64,float32,int64,int64,int64,float64,float32,float64,float32,str21
1,05 27 16.157,42 17 47.046,0.25712579,0.0240,0,0,0,-0.068,0.028,-0.170,0.019,195365594010029440
1,05 24 16.850,42 18 23.985,0.28059315,0.0219,0,0,0,0.160,0.025,-0.218,0.020,195052027036610048
1,05 21 08.064,43 04 32.745,0.28114515,0.0192,0,0,0,0.476,0.023,-0.550,0.016,207333812437636608
1,05 26 37.522,43 01 10.963,0.25281930,0.0176,0,0,0,0.138,0.024,0.311,0.017,195645832035222400
1,05 29 24.882,43 06 59.433,0.24000569,0.0188,0,0,0,0.288,0.021,-0.305,0.015,195474273863568000
1,05 24 00.710,42 25 08.388,0.38202042,0.0213,0,0,0,0.857,0.028,-2.544,0.020,195241619775197824
1,05 23 27.211,42 23 29.735,0.26806213,0.0206,0,0,0,0.123,0.026,-0.207,0.019,195243097243949184
1,05 22 36.398,42 24 04.222,0.38906178,0.0163,0,0,0,-0.239,0.020,-1.693,0.017,195232003349318144


In [ ]:
execute_command('cd giessler_traceback ; examples/trace/traceback g166/trace.conf')

# testing

In [84]:
from astropy.table import Table
import astropy.units as u
import numpy as np

def find_neighbors_by_source_id(table, coord_column, source_column, source_id, max_sep, return_table=False):
    """
    Find stars within a specified angular separation from a given source ID in an astropy table.
    The output is sorted by separation, with the target source as the first row.

    Parameters:
    -----------
    table : astropy.table.Table
        The input table containing sky coordinates and source IDs.
    coord_column : str
        The column name of the SkyCoord objects in the table.
    source_column : str
        The column name of the source IDs in the table.
    source_id : int or str
        The source ID of the star to search around.
    max_sep : astropy.units.Quantity
        The maximum separation angle within which to find neighbors.
    return_table : bool, optional
        If True, returns a table of the neighbors with the target source as the first row.
        If False, returns a list of source IDs.

    Returns:
    --------
    neighbors : list of int or astropy.table.Table
        Either a list of source IDs of neighboring stars or a table of these neighbors,
        with the target star as the first row.
    """

    # Ensure max_sep is an astropy Quantity with angular units
    if not isinstance(max_sep, u.Quantity):
        raise ValueError("max_sep must be an astropy.units.Quantity with angular units (e.g., degrees).")

    # Locate the index of the target star with the given source_id
    target_indices = np.where(table[source_column] == source_id)[0]
    if len(target_indices) == 0:
        raise ValueError(f"Source ID {source_id} not found in the table.")
    target_index = target_indices[0]

    # Get the SkyCoord object of the star at the specified index
    target_coord = table[coord_column][target_index]

    # Calculate separations between the target and all other coordinates in the table
    separations = target_coord.separation(table[coord_column])

    # Find indices where separation is within the max_sep, including the target itself
    neighbor_indices = np.where(separations < max_sep)[0]

    # Sort the neighbors by separation
    sorted_indices = neighbor_indices[np.argsort(separations[neighbor_indices])]

    # Get the source IDs of the sorted neighboring stars
    neighbor_source_ids = table[source_column][sorted_indices].tolist()

    # Return results based on the specified return type
    if return_table:
        # Create a subtable with the sorted neighbors' source IDs
        neighbor_table = table[sorted_indices]
        return neighbor_table
    else:
        return neighbor_source_ids


In [86]:
neighbor_source_ids = find_neighbors_by_source_id(sir, "SkyCoord", "Source", source_id=195633325090480896, max_sep=3*u.arcmin, return_table=True)
# print("Neighbor source IDs:", neighbor_source_ids)
# fs4giesler(neighbor_source_ids)
neighbor_source_ids

RA_ICRS_1,DE_ICRS_1,e_RA_ICRS,e_DE_ICRS,_r_1,HIP,TYC2,Source,rgeo,Plx,e_Plx,pmRA,pmDE,e_pmRA,e_pmDE,RUWE,Teff,logg,Gmag,BP-RP,BPmag,RPmag,RV,e_RV,b_rgeo,B_rgeo,FG,e_FG,FBP,e_FBP,FRP,e_FRP,RAVE5,RAVE6,e_Gmag,e_BPmag,e_RPmag,e_BP-RP,SkyCoord
deg,deg,mas,mas,,,,,pc,mas,mas,mas / yr,mas / yr,mas / yr,mas / yr,,K,log(cm.s**-2),mag,mag,mag,mag,km / s,km / s,pc,pc,,,,,,,,,,,,,"deg,deg,pc"
float64,float64,float64,float64,float64,int32,str11,int64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float64,float64,float64,float64,float64,float64,float32,float64,float64,float64,float32,float32,float32,float32,float32,str1,str1,float64,float32,float32,float32,SkyCoord
81.62564389768,42.93974983019,0.0447,0.0294,0.006438,--,--,195633325090480896,3138.43115000,0.2922,0.0544,1.408,-2.592,0.064,0.046,1.046,6047.5,4.4779,16.263456,1.037928,16.644405,15.606478,--,--,2643.20166000,3779.71533000,5882.52580,3.53,3004,17.82,4535,16.2,--,--,0.002831,0.00702,0.005416,0.01244,"81.62564389768,42.93974983019,3138.43115"
81.62187230463,42.93624922588,0.0429,0.0280,0.003716,--,--,195633320791780608,3241.66699000,0.2980,0.0489,0.846,-3.886,0.059,0.045,1.021,6518.7,4.3925,16.193027,1.071283,16.641203,15.569920,--,--,2832.63281000,3757.81396000,6276.76299,2.736,3013,13.36,4690,11.55,--,--,0.002796,0.005564,0.004629,0.01019,"81.62187230463,42.93624922588,3241.66699"
81.61260055545,42.95209988890,0.0418,0.0272,0.020854,--,--,195645041761386112,2658.02783000,0.3465,0.0463,0.097,-0.625,0.059,0.043,0.970,6328.1,4.4236,16.233372,1.079447,16.688612,15.609165,--,--,2319.83838000,3126.79761000,6047.80561,2.333,2884,8.967,4524,9.515,--,--,0.002787,0.00438,0.004416,0.008796,"81.61260055545,42.9520998889,2658.02783"
81.64062967618,42.92659012262,0.0171,0.0115,0.013274,--,--,195633045913862784,3680.09912000,0.2328,0.0197,-0.214,0.303,0.025,0.019,1.056,5619.7,3.0466,14.356781,1.493430,15.045052,13.551621,-22.33,2.07,3433.33960000,3935.00684000,34059.19885,5.756,1.31e+04,17.9,3.01e+04,46.01,--,--,0.002761,0.00316,0.004128,0.007287,"81.64062967618,42.92659012262,3680.09912"
81.60423028846,42.92918404038,0.0192,0.0125,0.015768,--,--,195633290730748544,2703.84351000,0.3354,0.0222,0.748,-2.973,0.028,0.021,1.028,6754.7,3.6982,14.610037,1.051590,15.046293,13.994703,-6.74,6.20,2560.54053000,2889.21631000,26973.16616,5.26,1.309e+04,20.05,2.001e+04,17.12,--,--,0.002763,0.003248,0.003892,0.00714,"81.60423028846,42.92918404038,2703.84351"
81.59936703854,42.93817444048,0.0346,0.0232,0.019389,--,--,195645041761269248,3332.52466000,0.2771,0.0401,0.075,-2.519,0.050,0.039,0.931,5637.6,4.0132,15.895838,1.463045,16.559977,15.096931,--,--,2922.06055000,3991.81665000,8252.99539,2.695,3247,14.01,7251,10.17,--,--,0.002778,0.005454,0.004075,0.009529,"81.59936703854,42.93817444048,3332.52466"
81.59270938120,42.93048227897,0.0373,0.0241,0.023820,--,--,195633187651535488,3031.43726000,0.2978,0.0419,-0.486,-1.182,0.049,0.037,1.201,8558.9,4.1257,15.515886,0.915350,15.867992,14.952642,--,--,2696.21387000,3522.58398000,11710.95258,5.712,6141,12.14,8281,11.93,--,--,0.002806,0.00352,0.00409,0.007611,"81.5927093812,42.93048227897,3031.43726"
81.58889547044,42.95540259001,0.0358,0.0238,0.034440,--,--,195645003105955712,3142.18481000,0.3008,0.0422,1.475,-0.452,0.053,0.039,0.988,--,--,16.056543,1.228042,16.586267,15.358226,--,--,2784.35791000,3813.83911000,7117.53739,2.458,3169,7.413,5700,9.38,--,--,0.002781,0.003773,0.00418,0.007953,"81.58889547044,42.95540259001,3142.18481"
